In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit



In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def random_forest_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    n_estimators=100,
    max_depth=None,
    random_state=42,
    use_cv=False,
    cv_params=None,
    cv=5
):
    """
    Random Forest regression for time series forecasting with lag features only.
    Supports optional hyperparameter tuning via time-series-aware cross-validation.
    """
    # 1. Directory structure
    os.makedirs('RandomForestRegression', exist_ok=True)
    
    # 2. Stationarity and Differencing
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 3. Feature Engineering - Lag features only
    df = pd.DataFrame({'y': series.astype(float)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(float)
    df = df.dropna()

    # 4. Train-Test Split (time-based)
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 5. Normalize features (fit on train only, apply to test)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 6. Model Training (with optional CV)
    if use_cv:
        param_grid = cv_params or {
            'n_estimators': [100, 200],
            'max_depth': [None, 5, 10, 20],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }
        tscv = TimeSeriesSplit(n_splits=cv)
        grid_search = GridSearchCV(
            RandomForestRegressor(random_state=random_state),
            param_grid,
            cv=tscv,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        grid_search.fit(X_train_scaled, y_train)
        model = grid_search.best_estimator_
        best_params = grid_search.best_params_
    else:
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state
        )
        model.fit(X_train_scaled, y_train)
        best_params = {
            'n_estimators': n_estimators,
            'max_depth': max_depth
        }

    # 7. Forecasting
    pred_test = model.predict(X_test_scaled)

    # 8. Inverse differencing if applied
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 9. Create forecast DataFrame
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test
    })

    # 10. Save forecasts to CSV
    forecast_csv_path = os.path.join('RandomForestRegression', 'rf_forecasts.csv')
    forecast_df.to_csv(
        forecast_csv_path,
        mode='a',
        header=not os.path.exists(forecast_csv_path),
        index=False
    )

    # 11. Calculate metrics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    r_squared = model.score(X_train_scaled, y_train)
    metrics_data = {
        'district': district_name,
        'rmse': rmse,
        'r_squared': r_squared,
        'best_params': best_params,
        'differencing': d
    }
    metrics_df = pd.DataFrame([metrics_data])

    metrics_csv_path = os.path.join('RandomForestRegression', 'rf_metrics.csv')
    metrics_df.to_csv(
        metrics_csv_path,
        mode='a',
        header=not os.path.exists(metrics_csv_path),
        index=False
    )

    # 12. Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(original_series.index, original_series.values, label='Original Series')
    plt.plot(forecast_df['date'], forecast_df['forecast'], label='RF Forecast', color='forestgreen')
    plt.title(f'Random Forest Forecast for {district_name}\nRMSE: {rmse:.2f}, R²: {r_squared:.2f}')
    plt.legend()
    plt.savefig(os.path.join('RandomForestRegression', f'rf_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'best_params': best_params,
        'feature_importances': model.feature_importances_
    }


In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I48"].asfreq('MS')
        
        results = random_forest_regression_lags_only(
            ts,
            district,
            max_lags=5,
            use_cv=True,
            cv_params={
                'n_estimators': [100, 200],
                'max_depth': [None, 5, 10]
            },
            cv=3
        )

        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district       rmse  r_squared                            best_params  \
0  AHMEDNAGAR  83.175733   0.850234  {'max_depth': 5, 'n_estimators': 100}   

   differencing  
0             0  

=== Forecast Data ===
     district       date  actual    forecast
0  AHMEDNAGAR 2020-07-01   354.0  430.562718
1  AHMEDNAGAR 2020-08-01   391.0  402.030248
2  AHMEDNAGAR 2020-09-01   417.0  416.067506
3  AHMEDNAGAR 2020-10-01   343.0  433.104607
4  AHMEDNAGAR 2020-11-01   468.0  404.692601
=== Metrics ===
  district       rmse  r_squared                            best_params  \
0    AKOLA  88.487854   0.837731  {'max_depth': 5, 'n_estimators': 100}   

   differencing  
0             0  

=== Forecast Data ===
  district       date  actual    forecast
0    AKOLA 2020-07-01   407.0  438.828587
1    AKOLA 2020-08-01   306.0  441.895254
2    AKOLA 2020-09-01   504.0  451.563131
3    AKOLA 2020-10-01   493.0  462.548934
4    AKOLA 2020-11-01   514.0  456.392399
=== Metrics ===
   d

{'district': 'MUMBAI SUBURBAN',
 'forecast_df':           district       date  actual  forecast
 0  MUMBAI SUBURBAN 2021-02-01   607.0    743.75
 1  MUMBAI SUBURBAN 2021-03-01   618.0    846.38,
 'metrics_df':           district        rmse  r_squared  \
 0  MUMBAI SUBURBAN  188.225911    0.80591   
 
                                 best_params  differencing  
 0  {'max_depth': None, 'n_estimators': 100}             0  ,
 'best_params': {'max_depth': None, 'n_estimators': 100},
 'feature_importances': array([0.1832363 , 0.10818202, 0.4053542 , 0.09753387, 0.20569361])}